In [1]:
!pip install langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/Indian_independence_movement.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

46


In [4]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

Indian independence movement
The Indian Independence Movement was a series of historic events in South Asia with the ultimate aim
of ending British colonial rule. It lasted until 1947, when the Indian

{'producer': 'Skia/PDF m128', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'creationdate': '2024-12-16T12:13:11+00:00', 'title': 'Indian independence movement - Wikipedia', 'moddate': '2024-12-16T12:13:11+00:00', 'source': '/content/Indian_independence_movement.pdf', 'total_pages': 46, 'page': 0, 'page_label': '1'}


In [90]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

269

In [91]:
!pip install -qU langchain-huggingface

In [92]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-distilroberta-v1")

In [93]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1)

Generated vectors of length 768

[0.011785837821662426, 0.02715216763317585, -0.006809072103351355, 0.05964178219437599, 0.10289416462182999, 0.005167822353541851, -0.03296468034386635, 0.014794649556279182, 0.048582885414361954, -0.024443218484520912, -0.052192673087120056, -0.01325240358710289, -0.04117967188358307, 0.039585188031196594, 0.061721671372652054, -0.05321848392486572, 0.06395433098077774, 0.026178233325481415, 0.023485092446208, -0.03073945827782154, -0.04143061116337776, 0.02167350985109806, -0.019488468766212463, -0.014448628760874271, 0.07783444225788116, 0.03274645656347275, 0.04642306640744209, -0.02075904794037342, -0.028354981914162636, 0.044887445867061615, 0.0022504921071231365, 0.004986620973795652, 0.02744654007256031, -0.013472105376422405, 0.018450956791639328, -0.014897952787578106, -0.0356437973678112, -0.014498759992420673, -0.0012308848090469837, -0.010517731308937073, -0.024909131228923798, 0.012558088637888432, -0.004659480415284634, -0.072406917810440

In [94]:
!pip install -qU langchain-chroma

In [95]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [96]:
ids = vector_store.add_documents(documents=all_splits)

In [97]:
results = vector_store.similarity_search(
    "How many people were killed in Jallianwala Bagh massacre?"
)

print(results[0])

page_content='Sidney Rowlatt, best remembered
for his controversial presidency of
the Rowlatt Committee, a sedition
committee appointed in 1918 by the
British Indian Government to
evaluate the links between political
terrorism in India, the actions
indirectly led to the infamous
Jallianwala Bagh massacre of 1919.
The Martyrs' Well of Jallianwala
Bagh massacre, at Jallianwala
Bagh. 120 bodies were recovered
from this well as per inscription on
it.[91]
Mahatma Gandhi.
the harsher way.[92] A total
of 1,651 rounds were fired,
killing 379 people (as
according to an official
British commission; Indian' metadata={'creationdate': '2024-12-16T12:13:11+00:00', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/128.0.0.0 Safari/537.36', 'moddate': '2024-12-16T12:13:11+00:00', 'page': 19, 'page_label': '20', 'producer': 'Skia/PDF m128', 'source': '/content/Indian_independence_movement.pdf', 'start_index': 0, 'title': 'Indian independence movement - Wi